<a href="https://colab.research.google.com/github/Lamiaka/GNN_amld2022/blob/main/Solution_to_Create_Graphs_%7C_AMLD_GNNs_for_Structured_Data_Workshop_by_Volodya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# README

## A solution to the Exercise - Greating Graphs

This is an example colab for Graph Neural Networks for Structured Data Workshop @ AMLD by volodya@google.com

Link to this colab: https://bit.ly/gnn-str-ex-2-solution



In [ ]:
!pip install folium==0.8.3 "imgaug<0.2.7,>=0.2.5" spektral

!wget 'https://drive.google.com/uc?id=1m3ACccA3GLg5GFg0D9khUNnt_c10gBAK&export=download' -O stocks-with-label_3days.prq

# NOTE: When running for the first time, you will get an error asking to restart runtime - click `Restart Runtime` button to do so and rerun the cells.

     |████████████████████████████████| 631 kB 6.8 MB/s 
     |████████████████████████████████| 123 kB 12.0 MB/s 
     |████████████████████████████████| 14.8 MB 23.2 MB/s 
     |████████████████████████████████| 462 kB 41.3 MB/s 
     |████████████████████████████████| 495.0 MB 28 kB/s 
     |████████████████████████████████| 1.3 MB 47.0 MB/s 
     |████████████████████████████████| 463 kB 35.5 MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654017 sha256=6a327067215cfe0a52cafed4434128381733c14da0d797528e50aedb2d8b8aee
  Stored in directory: /root/.cache/pip/wheels/89/72/98/3ebfdba1069a9a8eaaa7ae7265cfd67d63ef0197aaee2e5f9c
Successfully built imgaug
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Suc

--2022-03-23 19:50:53--  https://drive.google.com/uc?id=1m3ACccA3GLg5GFg0D9khUNnt_c10gBAK&export=download
Resolving drive.google.com (drive.google.com)... 142.250.141.100, 142.250.141.138, 142.250.141.139, ...
Connecting to drive.google.com (drive.google.com)|142.250.141.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-5c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/15i4u9mentvl9h7l90dhfeh2ilt8bv2l/1648065000000/01333786639642341364/*/1m3ACccA3GLg5GFg0D9khUNnt_c10gBAK?e=download [following]
--2022-03-23 19:50:54--  https://doc-08-5c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/15i4u9mentvl9h7l90dhfeh2ilt8bv2l/1648065000000/01333786639642341364/*/1m3ACccA3GLg5GFg0D9khUNnt_c10gBAK?e=download
Resolving doc-08-5c-docs.googleusercontent.com (doc-08-5c-docs.googleusercontent.com)... 142.250.141.132, 2607:f8b0:4023:c0b::84
Connecting to doc-08-5c-docs.googleusercontent.com (doc-08-5c

In [ ]:
import tensorflow as tf
from datetime import date
import pandas as pd
import numpy as np
import os

from typing import Dict, List, Text

import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.preprocessing import StandardScaler

import spektral

import tqdm

pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
stocks_df = pd.read_parquet('stocks-with-label_3days.prq')

stocks_df.sample(5)

,ticker,Date,LABEL,High,Low,Close,Open,High_1,Low_1,Close_1,Open_1
154013,DIS,2017-09-12,1.01,94.89,93.71,94.44,94.15,94.56,93.54,93.67,94.45
78961,JNJ,1965-06-17,1.04,0.09,0.09,0.09,0.00,0.09,0.09,0.09,0.00
152843,DIS,2013-01-18,1.01,47.58,47.10,47.58,47.49,47.91,47.13,47.65,47.13
35883,HD,1995-11-03,0.99,5.83,5.72,5.76,5.76,5.74,5.42,5.74,5.42
166950,BABA,2020-10-26,1.01,315.46,303.20,306.87,309.92,310.33,304.06,309.92,307.27


## Split the dataset into train, valid and test

In [ ]:
ticker_column = pd.Categorical(stocks_df['ticker'])
stocks_df['ticker'] = ticker_column
stocks_df['ticker'] = stocks_df.ticker.cat.codes

split_config = {
    'test_dt': pd.Timestamp('2020-01-01'), # 2020 onwards goes into test.
    'valid_dt': pd.Timestamp('2015-01-01'),
}

test_df = stocks_df.query('Date >= @test_dt', local_dict=split_config).copy()
train_df = stocks_df.query('Date < @valid_dt', local_dict=split_config).copy()
valid_df = stocks_df.query('Date >= @valid_dt and Date < @test_dt', local_dict=split_config).copy()

print(f'Train: {len(train_df)} Valid: {len(valid_df)} Test: {len(test_df)}')

Train: 139260 Valid: 28934 Test: 8970


## Scale the numerical columns and add LABEL


In [ ]:
columns_to_scale = list(set(train_df.columns) - {'Date', 'ticker', 'LABEL'})

std_scaler = StandardScaler()

def convert_df(df, fit_scaler=False):
  df.fillna(-1, inplace=True)
  for c, d in zip(df.columns, df.dtypes):
    if d == np.float64:
      df[c].astype('float32', copy=False)
  
  scaler_func = std_scaler.fit_transform if fit_scaler else std_scaler.transform
  df.loc[:, columns_to_scale] = scaler_func(df[columns_to_scale])

  return df


train_df = convert_df(train_df, fit_scaler=True)
valid_df = convert_df(valid_df, fit_scaler=False)
test_df = convert_df(test_df, fit_scaler=False)

train_df.head(10)

,ticker,Date,LABEL,High,Low,Close,Open,High_1,Low_1,Close_1,Open_1
147348,5,1991-04-01,1.03,-0.36,-0.35,-0.35,-0.35,-0.35,-0.35,-0.35,-0.35
23857,13,2011-08-24,1.05,0.12,0.10,0.11,0.12,0.12,0.09,0.13,0.09
100623,12,2006-12-05,1.00,-0.07,-0.07,-0.07,-0.07,-0.07,-0.07,-0.07,-0.07
99527,12,2002-07-31,0.94,-0.20,-0.20,-0.20,-0.20,-0.19,-0.20,-0.19,-0.20
6641,16,1999-06-08,1.06,0.38,0.39,0.39,0.39,0.38,0.39,0.38,0.38
150940,5,2005-06-28,0.97,-0.08,-0.08,-0.08,-0.08,-0.08,-0.08,-0.08,-0.08
149439,5,1999-07-09,1.02,-0.08,-0.08,-0.08,-0.07,-0.08,-0.07,-0.07,-0.07
16626,19,2002-02-06,1.01,-0.31,-0.31,-0.31,-0.31,-0.31,-0.31,-0.31,-0.31
155209,10,1981-02-03,1.02,-0.46,-0.46,-0.46,-0.49,-0.47,-0.46,-0.47,-0.49
88989,9,2005-04-21,1.01,0.35,0.36,0.35,0.35,0.36,0.36,0.35,0.36


# Define StocksDataset

In [ ]:
feature_dtype = np.float32
feature_names = ['ticker', 'High', 'Low', 'Close', 'Open', 'High_1', 'Low_1', 'Close_1', 'Open_1']

def get_features(example: Dict):
  return np.array([example[name] for name in feature_names], dtype=feature_dtype)

'''
An alternative implementation to the one below, which is functioanaly equivalent and easier to 
understand but is extremely inefficient.

def get_other_nodes(example: Dict, tickers: List[int], keyed_df: pd.DataFrame):
  def get_snapshot(date: Text, ticker: int):
    try:
      return keyed_df.loc[date, ticker]
    except KeyError:
      return None # Some tickers might be missing for some days

  nodes = []
  for t in tickers:
    if t != example['ticker']:
      snapshot = get_snapshot(example['Date'], t)
      if snapshot is not None:
        nodes.append(get_features(snapshot))
  return nodes

def get_graph(example: Dict, tickers: List[int], keyed_df: pd.DataFrame):
  nodes = [
           get_features(example), # Always the first.
           *get_other_nodes(example, tickers, keyed_df),
  ]
      
  x = np.array(nodes, dtype=feature_dtype).reshape(len(nodes), len(feature_names))
  # All nodes are connected to node 0 (features).
  a = np.vstack([
                 np.ones((1, len(nodes))), 
                 np.zeros((len(nodes) - 1, len(nodes))),
  ])
  e = None # No edge properties.
  y = example['LABEL'] > 1.01
  return spektral.data.Graph(x=x, a=a, y=y, e=e)

class StocksDataset(spektral.data.Dataset):
  def __init__(self, df: pd.DataFrame, **kwargs):
    self.df = df
    super().__init__(**kwargs)

  def read(self):
    tickers = list(self.df.ticker.unique())
    keyed_df = self.df.set_index(['Date', 'ticker'], drop=False)
    result = [get_graph(example, tickers, keyed_df) 
              for example in tqdm.tqdm(self.df.to_dict('records'))]
    return result
'''

def get_graph(example: Dict, keyed_df: pd.DataFrame):
  example_ticker = example['ticker']
  example_date = example['Date']
  examples_at_date = keyed_df.loc[[example_date]]
  examples_at_date = examples_at_date.sort_values(
      by='ticker', key=lambda ticker: ticker != example_ticker)
  
  nodes = [get_features(example) for example in examples_at_date.to_dict('records')]
      
  x = np.array(nodes, dtype=feature_dtype).reshape(len(nodes), len(feature_names))
  # All nodes are connected to node 0 (features).
  a = np.vstack([
                 np.ones((1, len(nodes))), 
                 np.zeros((len(nodes) - 1, len(nodes))),
  ])
  e = None # No edge features.
  y = example['LABEL'] > 1.01
  return spektral.data.Graph(x=x, a=a, y=y, e=e)

class StocksDataset(spektral.data.Dataset):
  def __init__(self, df: pd.DataFrame, **kwargs):
    self.df = df
    super().__init__(**kwargs)

  def read(self):
    keyed_df = self.df.set_index(['Date'], drop=False)
    result = [get_graph(example, keyed_df) for example in tqdm.tqdm(self.df.to_dict('records'))]
    return result

train_ds = StocksDataset(train_df.tail(20000))
display(train_ds)

valid_ds = StocksDataset(valid_df.tail(10000))

print('Element 0:')

display(train_ds[0].x)
display(train_ds[0].a)

100%|██████████| 20000/20000 [01:59<00:00, 168.06it/s]


StocksDataset(n_graphs=20000)

100%|██████████| 10000/10000 [00:44<00:00, 224.91it/s]

Element 0:


array([[ 5.        , -0.13475956, -0.16012122, -0.13788772, -0.15932983,
        -0.16217388, -0.16259225, -0.1631808 , -0.16041042],
       [ 9.        ,  0.19871637,  0.20022704,  0.20510918,  0.2062836 ,
         0.20177215,  0.20472611,  0.20035422,  0.20304635],
       [12.        , -0.1841997 , -0.18944101, -0.1903041 , -0.19115326,
        -0.19320543, -0.19329101, -0.1948693 , -0.18877324]],
      dtype=float32)

array([[1., 1., 1.],
       [0., 0., 0.],
       [0., 0., 0.]])

## Train

In [ ]:
from spektral.models.general_gnn import GeneralGNN
from spektral.data.loaders import DisjointLoader
from tensorflow.keras.losses import BinaryCrossentropy

batch_size = 1024

model = GeneralGNN(
    train_ds.n_labels, 
    hidden_activation='relu', 
    dropout=.2, 
    hidden=16, 
    pool='avg', 
    aggregate='mean',
    activation='sigmoid', 
    batch_norm=False)

model.compile(optimizer='adam', loss=BinaryCrossentropy(from_logits=False))

train_loader = DisjointLoader(train_ds, batch_size=batch_size)

valid_loader = DisjointLoader(valid_ds, batch_size=batch_size)

model.fit(
    train_loader.load(),
    steps_per_epoch=train_loader.steps_per_epoch,
    validation_data=valid_loader.load(),
    validation_steps=valid_loader.steps_per_epoch,
    epochs=2)

Epoch 1/2


 34%|███▍      | 1706/5000 [00:16<00:32, 101.17it/s]


20/20 [==============================] - 13s 547ms/step - loss: 0.7272 - val_loss: 0.6496
Epoch 2/2
20/20 [==============================] - 11s 529ms/step - loss: 0.6747 - val_loss: 0.6346


## Evaluate on test

In [ ]:
test_ds = StocksDataset(test_df.tail(5000))
test_loader = DisjointLoader(test_ds, batch_size=batch_size)

model.evaluate(test_loader.load(), steps=test_loader.steps_per_epoch)

100%|██████████| 5000/5000 [00:18<00:00, 263.81it/s]


5/5 [==============================] - 2s 360ms/step - loss: 0.6671


0.6671180725097656